## Quantitative value investing (Systematic value investing)
- Form of value investing that analyses fundamnetal data such as financial statement line items, economic data, unstructured data
- Portfolio diversification and rebalancing method 
- Ranking the market and setting buy/sell rules
- This project will build an investing strategy that selects the 50 stocks with the best value metrics
- This will determine and calculate the recommended trades for an equal-weight portfolio of these 50 stocks

In [9]:
import numpy as np 
import pandas as pd 
import xlsxwriter 
import requests
from scipy import stats
import math 

In [10]:
stocks = pd.read_csv('starter/sp_500_stocks.csv')
from starter.secrets import IEX_CLOUD_API_TOKEN

In [11]:
def chunks(lst, n):
    for i in range(0, len(lst),n):
        yield lst[i:i + n]
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

columns = ['Ticker', 'Price', 'Price-to-earnings ratio', 'Number of shares to buy']


In [12]:
final_df = pd.DataFrame(columns=columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data)
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
        pd.Series(
        [
            symbol, 
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A',
        ],
        index=columns
        ),
        ignore_index=True
        )
final_df 

,Ticker,Price,Price-to-earnings ratio,Number of shares to buy
0,A,151.11,50.5,N/A
1,AAL,23.08,-1.54,N/A
2,AAP,198.25,21.27,N/A
3,AAPL,136.93,30.43,N/A
4,ABBV,118.29,40.04,N/A
...,...,...,...,...
500,YUM,117.99,31.28,N/A
501,ZBH,160.39,60.91,N/A
502,ZBRA,495.91,42.96,N/A
503,ZION,51.80,6.27,N/A


## Removing glamour stocks 
- Opposite of value stock 
- Since the goal of this strategy is to identify the 50 best value stocks, we want to remove glamour stocks from the dataframe 
- Sort the dataframe by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50

In [ ]:
final_df.sort_values('Price-to-earnings ratio',inplace=True)
final_df = final_df[final_df['Price-to-earnings ratio'] > 0]
final_df = final_df[:50]
final_df.reset_index(inplace=True)
final_df.drop('index', axis=1, inplace=True)
final_df

In [21]:
# calculating number of shares to buy 
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:\n')
    
    try:
        val = float(portfolio_size)
        
    except ValueError:
        print('That is not a number')
        portfolio_size = input('Enter the value of your portfolio: \n')

Using the `portfolio_input` function to accept `portfolio_size` variable from the user of this script

In [22]:
portfolio_input()


Enter the value of your portfolio:
1000000000


In [25]:
position_size = float(portfolio_size)/len(final_df.index)
position_size
for row in final_df.index:
    final_df.loc[row, 'Number of shares to buy'] = math.floor(position_size/final_df.loc[row, 'Price'])
final_df

,Ticker,Price,Price-to-earnings ratio,Number of shares to buy
0,BIO,631.50,4.64,31670
1,CINF,119.25,6.13,167714
2,ZION,51.80,6.27,386100
3,AFL,52.94,6.96,377786
4,UNM,28.13,7.12,710984
5,HRB,24.11,7.91,829531
6,COO,392.38,8.27,50970
7,CE,148.81,8.46,134399
8,PGR,93.56,8.58,213766
9,KIM,20.74,8.72,964320


## Building a realistic value strategy 
- Price to earnings ratio doesn't work well with stocks with negative earnings 
- Stocks that buy back their own shares are difficult to value using `price-to-book` ratio
- Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies.
- We filter stocks with lowest percentiles on the following metrics:
    - Price-to-earnings ratio 
    - Price-to-book ratio 
    - Price-to-sales ratio 
    - Enterprise value divided by earnings before interest, taxes, depreciation, and amortization (EV/EBITDA)
    - Enterprise value divided by gross profict (EV/GP)
    

In [51]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data[symbol]['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio 
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise value divided by EBITDA
ev_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = ev_value/ebitda
print(ev_to_ebitda)

# Enterprise value divided by GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = ev_value/gross_profit
print(ev_to_gross_profit)


23.17126572787045
17.30100057005678


In [66]:
# robust value
rv_columns = [
    'Ticker',
    'Price',
    'Number of shares to buy',
    'Price-to-earnings ratio',
    'PE percentile',
    'Price-to-book ratio',
    'PB percentile',
    'Price-to-sales ratio',
    'PS percentile',
    'EV/EBITDA',
    'EV/EBITDA percentile',
    'EV/GP',
    'EV/GP percentile',
    'RV score ',
]

rv_df = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        ev_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
            
        rv_df = rv_df.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A',
        ],
        index=rv_columns),
            ignore_index=True
        )
        
        
rv_df
    

,Ticker,Price,Number of shares to buy,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,RV score
0,A,146.60,N/A,49.1,N/A,9.16,N/A,7.74,N/A,NaN,N/A,17.301001,N/A,N/A
1,AAL,22.72,N/A,-1.51,N/A,-1.9,N/A,1.14,N/A,NaN,N/A,17.301001,N/A,N/A
2,AAP,198.50,N/A,20.88,N/A,3.77,N/A,1.22,N/A,NaN,N/A,17.301001,N/A,N/A
3,AAPL,130.81,N/A,30.03,N/A,32.33,N/A,6.88,N/A,NaN,N/A,17.301001,N/A,N/A
4,ABBV,117.46,N/A,40.42,N/A,14.97,N/A,4.14,N/A,NaN,N/A,17.301001,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.10,N/A,30.96,N/A,-4.34,N/A,5.97,N/A,NaN,N/A,NaN,N/A,N/A
501,ZBH,163.73,N/A,58.74,N/A,2.79,N/A,4.73,N/A,NaN,N/A,NaN,N/A,N/A
502,ZBRA,512.67,N/A,42.8,N/A,11.19,N/A,5.71,N/A,NaN,N/A,NaN,N/A,N/A
503,ZION,51.40,N/A,6.06,N/A,1.15,N/A,2.44,N/A,NaN,N/A,NaN,N/A,N/A


## Dealing with mi